# Introduction
## Collaborative filtering

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. spark.ml currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries. spark.ml uses the alternating least squares (ALS) algorithm to learn these latent factors. The implementation in spark.ml has the following parameters:

numBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).

rank is the number of latent factors in the model (defaults to 10).

maxIter is the maximum number of iterations to run (defaults to 10).

regParam specifies the regularization parameter in ALS (defaults to 1.0).

implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).

alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).

nonnegative specifies whether or not to use nonnegative constraints for least squares (defaults to false).

### Explicit vs. implicit feedback

The standard approach to matrix factorization-based collaborative filtering treats the entries in the user-item matrix as explicit preferences given by the user to the item, for example, users giving ratings to movies.

It is common in many real-world use cases to only have access to implicit feedback (e.g. views, clicks, purchases, likes, shares etc.). The approach used in spark.ml to deal with such data is taken from Collaborative Filtering for Implicit Feedback Datasets. Essentially, instead of trying to model the matrix of ratings directly, this approach treats the data as numbers representing the strength in observations of user actions (such as the number of clicks, or the cumulative duration someone spent viewing a movie). Those numbers are then related to the level of confidence in observed user preferences, rather than explicit ratings given to items. The model then tries to find latent factors that can be used to predict the expected preference of a user for an item.

### Scaling of the regularization parameter

We scale the regularization parameter regParam in solving each least squares problem by the number of ratings the user generated in updating user factors, or the number of ratings the product received in updating product factors. This approach is named “ALS-WR” and discussed in the paper “Large-Scale Parallel Collaborative Filtering for the Netflix Prize”. It makes regParam less dependent on the scale of the dataset, so we can apply the best parameter learned from a sampled subset to the full dataset and expect similar performance.


### Cold-start strategy

When making predictions using an ALSModel, it is common to encounter users and/or items in the test dataset that were not present during training the model. This typically occurs in two scenarios:

In production, for new users or items that have no rating history and on which the model has not been trained (this is the “cold start problem”).
During cross-validation, the data is split between training and evaluation sets. When using simple random splits as in Spark’s CrossValidator or TrainValidationSplit, it is actually very common to encounter users and/or items in the evaluation set that are not in the training set
By default, Spark assigns NaN predictions during ALSModel.transform when a user and/or item factor is not present in the model. This can be useful in a production system, since it indicates a new user or item, and so the system can make a decision on some fallback to use as the prediction.

However, this is undesirable during cross-validation, since any NaN predicted values will result in NaN results for the evaluation metric (for example when using RegressionEvaluator). This makes model selection impossible.

Spark allows users to set the coldStartStrategy parameter to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values. The evaluation metric will then be computed over the non-NaN data and will be valid. Usage of this parameter is illustrated in the example below.

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

In [2]:
# Load the data 
df = pd.read_csv('steam-200k-cleaned.csv')
df = df.drop(['purchase','play'], axis=1)
df

,user,game,hours
0,5250,Alien Swarm,4.9
1,5250,Cities Skylines,144.0
2,5250,Counter-Strike,0.0
3,5250,Counter-Strike Source,0.0
4,5250,Day of Defeat,0.0
...,...,...,...
128799,309626088,Age of Empires II HD Edition,6.7
128800,309812026,Counter-Strike Nexon Zombies,0.0
128801,309812026,Robocraft,0.0
128802,309824202,Dota 2,0.7


In [3]:
# Drop NaN columns
data = df.dropna()
data = df.copy()

In [4]:
# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['game'] = data['game'].astype("category")
data['user_id'] = data['user'].cat.codes
data['game_id'] = data['game'].cat.codes

In [5]:
data

,user,game,hours,user_id,game_id
0,5250,Alien Swarm,4.9,0,227
1,5250,Cities Skylines,144.0,0,852
2,5250,Counter-Strike,0.0,0,978
3,5250,Counter-Strike Source,0.0,0,984
4,5250,Day of Defeat,0.0,0,1131
...,...,...,...,...,...
128799,309626088,Age of Empires II HD Edition,6.7,12389,174
128800,309812026,Counter-Strike Nexon Zombies,0.0,12390,982
128801,309812026,Robocraft,0.0,12390,3562
128802,309824202,Dota 2,0.7,12391,1336


In [6]:
import os
os.environ['SPARK_HOME'] = 'C:\Spark\spark-3.0.1-bin-hadoop2.7'
os.environ['HADOOP_HOME'] = 'C:\Spark\spark-3.0.1-bin-hadoop2.7'
# os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_271'
# print(os.environ['SPARK_HOME'])
# print(os.environ['HADOOP_HOME'])
# print(os.environ['JAVA_HOME'])
# print(os.environ['PATH'])

In [7]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.0.1-bin-hadoop2.7'

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

data = spark.createDataFrame(data)

In [9]:
(train, test) = data.randomSplit([0.8, 0.2])

In [10]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# If the rating matrix is derived from another source of information (i.e. it is inferred from other signals),
# you can set implicitPrefs to True to get better results:

als = ALS(seed = 123, implicitPrefs=True, nonnegative = True,
          userCol="user_id", itemCol="game_id", ratingCol="hours",coldStartStrategy="drop")

In [12]:
# Hyperparameter tuning and cross validation

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Add hyperparameters and their respective values to param_grid

param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .addGrid(als.maxIter, [5, 10, 15, 20])  \
            .build()

In [13]:
# Define evaluator as RMSE and print length of evaluator

evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="hours", 
           predictionCol="prediction") 

print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  160


In [14]:
# Build cross validation using CrossValidator

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

In [16]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

161.51516045926317


In [17]:
print("**Best Model**")

# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 10
  MaxIter: 20
  RegParam: 0.01


In [18]:
test_predictions.cache().show()

+---------+------------------+-----+-------+-------+------------+
|     user|              game|hours|user_id|game_id|  prediction|
+---------+------------------+-----+-------+-------+------------+
|174276289|    Battle Nations|  0.3|   6706|    471|         0.0|
|108183518|    Battle Nations|  0.0|   3573|    471|         0.0|
| 72978546|    Battle Nations|  0.0|   2081|    471| 0.017364305|
|166669789|    Battle Nations|  0.4|   6297|    471|0.0028338232|
| 87694960|       Bejeweled 3|  0.0|   2659|    496|  0.14125273|
| 59825286|       Bejeweled 3|  1.7|   1621|    496| 0.094944015|
| 88035166|       Bejeweled 3|  0.0|   2675|    496|  0.10140009|
|117531196|       Bejeweled 3|  0.0|   3978|    496|  0.20194621|
| 59332369|       Bejeweled 3|  0.0|   1605|    496| 0.036563497|
|116515230|       Bejeweled 3|  0.0|   3934|    496|         0.0|
| 59662830|       Bejeweled 3|  0.0|   1614|    496|  0.12550147|
| 22301321|       Bejeweled 3|  1.7|    427|    496|  0.24820733|
| 83090686

In [19]:
# Generate top 10 game recommendations for each user

userRecs = best_model.recommendForAllUsers(10)
userRecs.show(truncate = False)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                                                                                                                   |
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1580   |[[3825, 1.4440615], [4885, 0.9810278], [329, 0.9517418], [3570, 0.9429007], [4314, 0.9401218], [2366, 0.9400289], [1552, 0.9229098], [3223, 0.91871667], [326, 0.908479], [1133, 0.8773182]]                      |
|5300   |[[1336, 0.66333973], [4257, 0.59990484], [1762, 0.27640787], [1765, 0.23461501], [1761, 0.23296341], [1760,

In [20]:
# Generate top 10 user recommendations for each game

gameRecs = best_model.recommendForAllItems(10)
gameRecs.show(truncate = False)

+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|game_id|recommendations                                                                                                                                                                                                           |
+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1580   |[[1729, 0.13320497], [728, 0.12741907], [2155, 0.11961811], [1191, 0.11785227], [473, 0.10555494], [191, 0.10539895], [4162, 0.104263425], [2860, 0.1034668], [1107, 0.102071606], [1187, 0.10193567]]                    |
|4900   |[[728, 1.5214993], [2155, 1.262293], [1895, 1.2613652], [1729, 1.1893626], 

In [21]:
# Generate top 10 game recommendations for a specified set of users

users = data.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)
userSubsetRecs.show(truncate = False)

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                                                                                                     |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|26     |[[978, 0.21707082], [981, 0.21153267], [984, 0.19746456], [4788, 0.16979359], [2475, 0.16964744], [1336, 0.15337446], [1894, 0.15268216], [979, 0.08811954], [2807, 0.08768034], [3065, 0.07799246]]|
|474    |[[0, 0.0], [10, 0.0], [20, 0.0], [30, 0.0], [50, 0.0], [60, 0.0], [70, 0.0], [80, 0.0], [90, 0.0], [100, 0.0]]                                                     

In [22]:
# Generate top 10 user recommendations for a specified set of games

games = data.select(als.getItemCol()).distinct().limit(3)
gameSubSetRecs = best_model.recommendForItemSubset(games, 10)
gameSubSetRecs.show(truncate = False)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|game_id|recommendations                                                                                                                                                                                 |
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2040   |[[1048, 0.7914508], [211, 0.5755134], [530, 0.57310236], [1729, 0.57087266], [1191, 0.54562414], [1390, 0.5381665], [1279, 0.5341244], [1646, 0.53318286], [864, 0.53174835], [148, 0.52893734]]|
|2529   |[[1048, 0.6473613], [1729, 0.55844474], [1191, 0.5378163], [532, 0.5116988], [3277, 0.4905089], [1105, 0.49012423], [473, 0.4885398], [530, 0.48213467], [1235, 0.47771454], [346, 

In [24]:
# Evaluate game prediction
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation

rating_eval = SparkRatingEvaluation(test, test_predictions, col_user="user_id", col_item="game_id", 
                                    col_rating="hours", col_prediction="prediction")
rank_eval = SparkRankingEvaluation(test, test_predictions, col_user="user_id", col_item="game_id", 
                                    col_rating="hours", col_prediction="prediction")

print("Model:\tALS game prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(),
      "precision:\t%f" % rank_eval.precision_at_k(),
      "recall:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS game prediction
RMSE:	161.515160
MAE:	24.526275
Explained variance:	0.000531
R squared:	-0.022804
precision:	0.338325
recall:	0.943998
